In [1]:
import geopandas as gpd
from pathlib import Path
import xarray as xr
import numpy as np
import pandas as pd

import cdsapi
import matplotlib.pyplot as plt
import cartopy.crs as crs
import seaborn as sns

import os
import subprocess
from functools import partial
from collections import namedtuple
from pprint import pprint

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 6

In [2]:
# Set working directory as project root
if os.getcwd().split('/')[-1] != "ESoWC_dummy":
    os.chdir('..')
    
assert os.getcwd().split('/')[-1] == "ESoWC_dummy", f"Working directory should be the root (), currently: {os.getcwd()}"
print("Current Directory: ",os.getcwd())

Current Directory:  /Users/tommylees/github/ESoWC_dummy


In [3]:
# Set data_dir to external if on tommy's laptop (no space!)
if os.getcwd().split('/')[2] == 'tommylees':
    data_dir = Path("/Volumes/Lees_Extend/data/ecmwf_sowc/")
else:
    data_dir = Path('./data')
    
print(data_dir)
pprint([f.as_posix().split('/')[-1] for f in data_dir.glob('*')])

/Volumes/Lees_Extend/data/ecmwf_sowc
['chirps_kenya.nc',
 'chirps_kenya2.nc',
 'countries_shp',
 'countries_shp.zip',
 'era5_test.grib',
 'ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7b.nc',
 'ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7b.nc.zip',
 'ESACCI-LC-Legend.csv',
 'GRUN_v1_GSWP3_WGS84_05_1902_2014.nc',
 'kenya_precip_JanJune.grib',
 'kenya_rainfall2.nc',
 'kenya_rainfall_.nc',
 'orography.grib',
 'pressure.grib']


# Working with the spi functions

In [7]:
from climate_indices.__main__ import _compute_write_index

In [ ]:
netcdf_precip = _prepare_file(arguments.netcdf_precip,
                              arguments.var_name_precip)

# keyword arguments used for the SPI function
kwrgs = {"index": "spi",
         "netcdf_precip": netcdf_precip,
         "var_name_precip": arguments.var_name_precip,
         "input_type": input_type,
         "scale": scale,
         "distribution": dist,
         "periodicity": arguments.periodicity,
         "calibration_start_year": arguments.calibration_start_year,
         "calibration_end_year": arguments.calibration_end_year,
         "output_file_base": arguments.output_file_base}

_compute_write_index(kwrgs)

def _spi(precips, parameters):
    return indices.spi(values=precips,
                       scale=parameters["scale"],
                       distribution=parameters["distribution"],
                       data_start_year=parameters["data_start_year"],
                       calibration_year_initial=parameters["calibration_year_initial"],
                       calibration_year_final=parameters["calibration_year_final"],
                       periodicity=parameters["periodicity"])


In [10]:
args = {"index": "spi ",
"periodicity": "monthly ",
"netcdf_precip": "chirps_kenya2.nc",
"var_name_precip": "precip",
"output_file_base": "./kenya_monthly_spi",
"scales": [1,3,6],
"calibration_start_year": 1980,
"calibration_end_year": 2017,
"multiprocessing": "all",
}

In [12]:
ds = xr.open_dataset(data_dir/args["netcdf_precip"])
ds

<xarray.Dataset>
Dimensions:  (lat: 45, lon: 35, time: 1430)
Coordinates:
  * time     (time) datetime64[ns] 1900-01-01 1900-02-01 ... 2019-02-01
  * lon      (lon) float32 33.625 33.875 34.125 34.375 ... 41.625 41.875 42.125
  * lat      (lat) float32 -5.125 -4.875 -4.625 -4.375 ... 5.375 5.625 5.875
Data variables:
    precip   (time, lat, lon) float32 ...

In [8]:
from climate_indices import indices

CPUDispatcher(<function spi at 0x1c25c64d90>)

# When working with the command line arguments

## here are your options

```python
PARAMS:
-------
* "--index", help="Indices to compute", 
    * choices=["spi", "spei", "pnp", "scaled", "pet", "palmers", "all"],
* "--periodicity", help="Process input as either monthly or daily values", 
    * choices=[compute.Periodicity.monthly, * compute.Periodicity.daily],
* "--scales", help="Timestep scales over which the PNP, SPI, and SPEI values are to be computed",
* "--calibration_start_year", help="Initial year of the calibration period",
* "--calibration_end_year", help="Final year of calibration period", type=int
* "--netcdf_precip", help="Precipitation NetCDF file to be used as input for indices computations",
* "--var_name_precip", help="Precipitation variable name used in the precipitation NetCDF file",
* "--netcdf_temp", help="Temperature NetCDF file to be used as input for indices computations",
* "--var_name_temp", help="Temperature variable name used in the temperature NetCDF file",
* "--netcdf_pet", help="PET NetCDF file to be used as input for SPEI and/or Palmer computations",
* "--var_name_pet", help="PET variable name used in the PET NetCDF file"
* "--netcdf_awc", help="Available water capacity NetCDF file to be used as input for the Palmer computations",
* "--var_name_awc", help="Available water capacity variable name used in the AWC NetCDF file",
* "--output_file_base", help="Base output file path and name for the resulting output files",
* "--multiprocessing", help="Indices to compute", choices=["single", "all_but_one", "all"],

```

## and some [examples](https://climate-indices.readthedocs.io/en/latest/#getting-started)

```
$ process_climate_indices --index spi  --periodicity daily --netcdf_precip /data/cmorph_lowres_daily_conus_prcp.nc --var_name_precip prcp --output_file_base <out_dir>/cmorph_lowres_daily_conus --scales 30 90 --calibration_start_year 1998 --calibration_end_year 2016 --multiprocessing all

```

```
$ process_climate_indices --index spi --periodicity monthly --netcdf_precip /data/nclimgrid_lowres_prcp.nc --var_name_precip  prcp --output_file_base <out_dir>/nclimgrid_lowres --scales 6 12 --calibration_start_year 1951 --calibration_end_year 2010 --multiprocessing all

```